In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

EDA of Kickstarter Projects

Kickstarter is an American public-benefit corporation based in Brooklyn, New York, that maintains a global crowdfunding platform focused on creativity and merchandising. The company's stated mission is to "help bring creative projects to life"

In this exploratory data analysis I will try to find out a few interesting insights.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly
from plotly import tools
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../input/kickstarter-projects/ks-projects-201801.csv')
df

In [ ]:
df['deadline']=pd.to_datetime(df['deadline'], format="%Y/%m/%d").dt.date
df['launched']=pd.to_datetime(df['launched'], format="%Y/%m/%d").dt.date
df.head(5)

In [ ]:
df = df.drop(['ID','goal','pledged','usd_pledged_real'],1)# cleaninng data 
df['duration(days)'] = (df['deadline'] - df['launched']).dt.days # to count number of days taken 
df['launch_year']=pd.to_datetime(df['launched'], format="%Y/%m/%d").dt.year
df.head()

In [ ]:
df['state'].describe() # top is failed

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set(rc={'figure.figsize':[15,15]},font_scale=0.7)

Distribution of state

In [ ]:
sns.displot(df['state'],color='m')

Distribution of Main Categories

In [ ]:
df['main_category'].describe() #top is Film & Video

In [ ]:
sns.displot(df['main_category'],color='r')

In [ ]:
main_cat = df.main_category.value_counts()
go1 = go.Bar(
            x=main_cat.index,
            y=main_cat.values,
            marker=dict(
                color='rgb(158,202,225)',
                line=dict(
                    color='rgb(8,48,107)',
                    width=1.5),
            ),
            opacity=0.6
        )

data = [go1]
layout=go.Layout(title="Distribution of main_category", xaxis={'title':'main_category'}, yaxis={'title':'main_category'}, width=600, height=400)
figure=go.Figure(data=data,layout=layout)
iplot(figure)

Distribution of Categories

In [ ]:
sns.displot(df['category'])

of just 20 categories

In [ ]:
go1 = go.Bar(
            x=df.category.value_counts()[:20].index,
            y=df.category.value_counts()[:20].values,
            marker=dict(
                color='rgb(158,202,225)',
                line=dict(
                    color='rgb(8,48,107)',
                    width=1.5),
            ),
            opacity=0.6
        )
data = [go1]
layout=go.Layout(title="Distribution of Top 20 Categories", xaxis={'title':'Category'}, yaxis={'title':'No of Campaigns'}, width=600, height=400)
figure=go.Figure(data=data,layout=layout)
iplot(figure)

Distribution of Countries



In [ ]:
sns.displot(df['country'],color='g')

Campaign Distribution over Years



In [ ]:
df['launch_year']=pd.to_datetime(df['launched'], format="%Y/%m/%d").dt.year


In [ ]:
vs = sns.countplot(df.launch_year)
plt.xlabel("Year")
plt.ylabel("Number of Campaigns")
plt.title('"Number of Campaigns vs year')
plt.show(vs)

In [ ]:
df

Mean duration of Failed, Successful and Cancelled Campaigns



In [ ]:
failed = df.loc[df.state=='failed']
successful = df.loc[df.state=='successful']
canceled = df.loc[df.state=='canceled']
print('Mean duration of failed campaigns',failed['duration(days)'].mean())
print('Mean duration of successful campaigns',successful['duration(days)'].mean())
print('Mean duration of canceled campaigns',canceled['duration(days)'].mean())

In [ ]:
df

Distribution of main categories in Sucessful & Failed Campaigns



In [ ]:
trace1 = go.Bar(
            x=successful.category.value_counts()[:20].index,
            y=successful.category.value_counts()[:20].values,
            opacity=0.65
        )

trace2 = go.Bar(
            x=failed.category.value_counts()[:20].index,
            y=failed.category.value_counts()[:20].values,
            opacity=0.65
        )

fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('Successful', 'Failed'))
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout'].update(height=300, width=900, title='Distribution of top 20 categories in Sucessful & Failed Campaigns')

iplot(fig)

In [ ]:
successful.groupby(['main_category']).mean()


Relation between Backers and Pledged Amount(USD)



In [ ]:
df

In [ ]:
sns.regplot(x='backers',y='usd pledged', data=successful)


Relation between Duration(Days) and Pledged Amount(USD)



In [ ]:
sns.regplot(x='usd pledged',y='duration(days)', data=successful)


Success Measure

I have made another column called "success_measure" by dividing the Pledged Amount by the Goal of the Campaign which gives an idea of how many times the goal, the pledged amount was. But the issue was a lot of campaigns had weird goals like 10 or amounts such as 200. To consider only serious campaigns I eliminated any campaign whose goal was below the median goal of all the campaigns and after that I got a list of the 10 most successful campaigns in Kickstarter

In [ ]:
successful['success_measure'] = successful['usd pledged']/successful['usd_goal_real']
successful_cleaned = successful[successful['usd_goal_real']>successful['usd_goal_real'].median()]
successful_cleaned.nlargest(10,'success_measure')